## Preprocessing

In [24]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [25]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(columns=["EIN","NAME"])

In [26]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [27]:
# Look at APPLICATION_TYPE value counts for binning
application_df.groupby('APPLICATION_TYPE').count().iloc[:, 0]

APPLICATION_TYPE
T10      528
T12       27
T13       66
T14        3
T15        2
T17        1
T19     1065
T2        16
T25        3
T29        2
T3     27037
T4      1542
T5      1173
T6      1216
T7       725
T8       737
T9       156
Name: AFFILIATION, dtype: int64

In [28]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cut_off = 25
application_types_to_replace = []

bin_count= application_df.groupby('APPLICATION_TYPE').count().iloc[:, 0]

for x,i in bin_count.items():
  if i<cut_off:
    application_types_to_replace.append(x)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       27
T12         27
Name: count, dtype: int64

In [29]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [31]:
# You may find it helpful to look at CLASSIFICATION value counts >1
count_over_1 = application_df['CLASSIFICATION'].value_counts()
count_over_1[count_over_1>1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [33]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 500
classifications_to_replace = []
bin_count= application_df.groupby('CLASSIFICATION').count().iloc[:, 0]
for x,i in bin_count.items():
  if i<cutoff:
    classifications_to_replace.append(x)
# Replace in dataframe

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [34]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [35]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df = pd.get_dummies(application_df,columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT'])
dummy_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,...,SPECIAL_CONSIDERATIONS_Y,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,1,108590,1,False,False,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,False
2,1,5000,0,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,1,6692,1,False,False,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,False
4,1,142590,1,False,False,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
34295,1,5000,0,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
34296,1,5000,0,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
34297,1,5000,1,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [36]:
# Split our preprocessed data into our features and target arrays
y = dummy_df['IS_SUCCESSFUL']
x = dummy_df.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=78)

In [37]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(x_train)

# Scale the data
X_train_scaled = X_scaler.transform(x_train)
X_test_scaled = X_scaler.transform(x_test)

## Compile, Train and Evaluate the Model

In [38]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = 47
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
nn = tf.keras.models.Sequential()


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 384       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 435 (1.70 KB)
Trainable params: 435 (1.70 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

In [40]:
# Train the model

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints",exist_ok=True)
checkpoint_path = "checkpoints/Del_2_E{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5th epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq=804*5,
    save_weights_only=True,
    verbose=0)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=1000, callbacks=[cp_callback])



Epoch 1/1000
804/804 [==============================] - 1s 675us/step - loss: 0.6046 - accuracy: 0.6884
Epoch 2/1000
804/804 [==============================] - 1s 639us/step - loss: 0.5674 - accuracy: 0.7213
Epoch 3/1000
804/804 [==============================] - 1s 632us/step - loss: 0.5614 - accuracy: 0.7263
Epoch 4/1000
804/804 [==============================] - 1s 652us/step - loss: 0.5577 - accuracy: 0.7281
Epoch 5/1000
804/804 [==============================] - 1s 654us/step - loss: 0.5563 - accuracy: 0.7279
Epoch 6/1000
804/804 [==============================] - 1s 639us/step - loss: 0.5548 - accuracy: 0.7308
Epoch 7/1000
804/804 [==============================] - 1s 683us/step - loss: 0.5542 - accuracy: 0.7296
Epoch 8/1000
804/804 [==============================] - 1s 659us/step - loss: 0.5532 - accuracy: 0.7310
Epoch 9/1000
804/804 [==============================] - 1s 644us/step - loss: 0.5529 - accuracy: 0.7319
Epoch 10/1000
804/804 [==============================] - 1s 668u

In [41]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5554 - accuracy: 0.7248 - 235ms/epoch - 876us/step
Loss: 0.5554164052009583, Accuracy: 0.724781334400177


In [42]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

C:\Users\Leo\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
